In [ ]:
import torch
from torch import nn,optim
import functools
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torchvision.utils as vutils

In [ ]:
import random
import numpy as np
import torch.utils.data as data
from PIL import Image
import torchvision.transforms as transforms

In [ ]:
from tensorboardX import SummaryWriter
writer=SummaryWriter('/user1/faculty/cvpr/ujjwal/dhritimaan/datasets/log_cycle_gan/')

In [ ]:
class SequenceFolder(data.Dataset):
    def __init__(self, trainA,trainB):
        self.trainA=trainA
        self.trainB=trainB

    def __len__(self):
        return 400
    def __getitem__(self, index):
        imgA=plt.imread(self.trainA[index])
        imgB=plt.imread(self.trainB[index])
        imgA=plt.imread(self.trainA[index])
        imgB=plt.imread(self.trainB[index])
        transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])

        dict_img={'A':transform(imgA),'B':transform(imgB)}
        return dict_img

In [5]:
import os
a=os.listdir('/user1/faculty/cvpr/ujjwal/dhritimaan/datasets/facades/trainA')
b=os.listdir('/user1/faculty/cvpr/ujjwal/dhritimaan/datasets/facades/trainB')
lista=[]
listb=[]
for i in range(400):
    lista.append(os.path.join('/user1/faculty/cvpr/ujjwal/dhritimaan/datasets/facades/trainA',a[i]))
    listb.append(os.path.join('/user1/faculty/cvpr/ujjwal/dhritimaan/datasets/facades/trainB',b[i]))

In [6]:
train=SequenceFolder(lista,listb)

In [7]:
dataloader=torch.utils.data.DataLoader(train,batch_size=4,shuffle=True)

In [8]:
import matplotlib.pyplot as plt
for a,b in enumerate(dataloader):
    a
    break

In [9]:
b['A'].shape

torch.Size([4, 3, 256, 256])

In [10]:
b['B'].permute(0,3,1,2).shape

torch.Size([4, 256, 3, 256])

In [11]:
class ResnetBlock(nn.Module):
    """Define a Resnet block"""

    def __init__(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        """Initialize the Resnet block
        A resnet block is a conv block with skip connections
        We construct a conv block with build_conv_block function,
        and implement skip connections in <forward> function.
        Original Resnet paper: https://arxiv.org/pdf/1512.03385.pdf
        """
        super(ResnetBlock, self).__init__()
        self.conv_block = self.build_conv_block(dim, padding_type, norm_layer, use_dropout, use_bias)

    def build_conv_block(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        """Construct a convolutional block.
        Parameters:
            dim (int)           -- the number of channels in the conv layer.
            padding_type (str)  -- the name of padding layer: reflect | replicate | zero
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers.
            use_bias (bool)     -- if the conv layer uses bias or not
        Returns a conv block (with a conv layer, a normalization layer, and a non-linearity layer (ReLU))
        """
        conv_block = []
        p = 0
        if padding_type == 'reflect':
            conv_block += [nn.ReflectionPad2d(1)]
        elif padding_type == 'replicate':
            conv_block += [nn.ReplicationPad2d(1)]
        elif padding_type == 'zero':
            p = 1
        else:
            raise NotImplementedError('padding [%s] is not implemented' % padding_type)

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim), nn.ReLU(True)]
        if use_dropout:
            conv_block += [nn.Dropout(0.5)]

        p = 0
        if padding_type == 'reflect':
            conv_block += [nn.ReflectionPad2d(1)]
        elif padding_type == 'replicate':
            conv_block += [nn.ReplicationPad2d(1)]
        elif padding_type == 'zero':
            p = 1
        else:
            raise NotImplementedError('padding [%s] is not implemented' % padding_type)
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim)]

        return nn.Sequential(*conv_block)

    def forward(self, x):
        """Forward function (with skip connections)"""
        out = x + self.conv_block(x)  # add skip connections
        return out

In [12]:
class ResnetGenerator(nn.Module):
    """Resnet-based generator that consists of Resnet blocks between a few downsampling/upsampling operations.
    We adapt Torch code and idea from Justin Johnson's neural style transfer project(https://github.com/jcjohnson/fast-neural-style)
    """

    def __init__(self, input_nc, output_nc, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=False, n_blocks=6, padding_type='reflect'):
        """Construct a Resnet-based generator
        Parameters:
            input_nc (int)      -- the number of channels in input images
            output_nc (int)     -- the number of channels in output images
            ngf (int)           -- the number of filters in the last conv layer
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers
            n_blocks (int)      -- the number of ResNet blocks
            padding_type (str)  -- the name of padding layer in conv layers: reflect | replicate | zero
        """
        assert(n_blocks >= 0)
        super(ResnetGenerator, self).__init__()
        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        model = [nn.ReflectionPad2d(3),
                 nn.Conv2d(input_nc, ngf, kernel_size=7, padding=0, bias=use_bias),
                 norm_layer(ngf),
                 nn.ReLU(True)]

        n_downsampling = 2
        for i in range(n_downsampling):  # add downsampling layers
            mult = 2 ** i
            model += [nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3, stride=2, padding=1, bias=use_bias),
                      norm_layer(ngf * mult * 2),
                      nn.ReLU(True)]

        mult = 2 ** n_downsampling
        for i in range(n_blocks):       # add ResNet blocks

            model += [ResnetBlock(ngf * mult, padding_type=padding_type, norm_layer=norm_layer, use_dropout=use_dropout, use_bias=use_bias)]

        for i in range(n_downsampling):  # add upsampling layers
            mult = 2 ** (n_downsampling - i)
            model += [nn.ConvTranspose2d(ngf * mult, int(ngf * mult / 2),
                                         kernel_size=3, stride=2,
                                         padding=1, output_padding=1,
                                         bias=use_bias),
                      norm_layer(int(ngf * mult / 2)),
                      nn.ReLU(True)]
        model += [nn.ReflectionPad2d(3)]
        model += [nn.Conv2d(ngf, output_nc, kernel_size=7, padding=0)]
        model += [nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, input):
        """Standard forward"""
        return self.model(input)

In [13]:
class NLayerDiscriminator(nn.Module):
    """Defines a PatchGAN discriminator"""

    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d):
        """Construct a PatchGAN discriminator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            n_layers (int)  -- the number of conv layers in the discriminator
            norm_layer      -- normalization layer
        """
        super(NLayerDiscriminator, self).__init__()
        if type(norm_layer) == functools.partial:  # no need to use bias as BatchNorm2d has affine parameters
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        kw = 4
        padw = 1
        sequence = [nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw), nn.LeakyReLU(0.2, True)]
        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):  # gradually increase the number of filters
            nf_mult_prev = nf_mult
            nf_mult = min(2 ** n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]  # output 1 channel prediction map
        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        """Standard forward."""
        return self.model(input)

In [14]:
cuda=torch.device('cuda:1')
genAB=ResnetGenerator(3,3).to(cuda)
genBA=ResnetGenerator(3,3).to(cuda)

In [15]:
discAB=NLayerDiscriminator(3).to(cuda)
discBA=NLayerDiscriminator(3).to(cuda)

In [16]:
from itertools import chain

In [17]:
gen_optim=optim.Adam(chain(genAB.parameters(),genBA.parameters()),lr=0.0002,betas=(0.5,0.999))
dis_optim=optim.Adam(chain(discAB.parameters(),discBA.parameters()),lr=0.0002,betas=(0.5,0.999))

In [18]:
#loss_ce=nn.NLLLoss()
#loss_mse=nn.MSELoss()
#loss_rec=l1loss()

In [19]:
def l1loss(x,y):
    return torch.mean(torch.abs(x-y))
def loss_mse(x,y):
    return torch.mean((x-y)**2)

In [20]:
#nn.L1Loss()

In [21]:
num_epochs=100
count=0
for epoch in range(num_epochs):
    for n_batches,real_data in enumerate(dataloader):
        realA=real_data['A'].float().to(cuda)#.permute(0,3,1,2).float().to(cuda)#/255
        realB=real_data['B'].float().to(cuda)#.permute(0,3,1,2).float().to(cuda)#/255
        
        
        fakeB=genAB(realA)
        fakeA=genBA(realB)
        recA=genBA(fakeB)
        recB=genAB(fakeA)
        dis_real_labB=discAB(realB)
        dis_fake_labB=discAB(fakeB)
        dis_real_labA=discBA(realA)
        dis_fake_labA=discBA(fakeA)
        dis_rec_labA=discBA(recA)
        dis_rec_labB=discAB(recB)
        
        gen_optim.zero_grad()
        #######training generators
        error_genAB=loss_mse(dis_fake_labB,Variable(torch.ones(realA.shape[0],1,30,30).to(cuda)))
        error_genBA=loss_mse(dis_fake_labA,Variable(torch.ones(realA.shape[0],1,30,30).to(cuda)))
        rec_errorA=l1loss(recA,realA)#loss_rec(recA,realA)
        rec_errorB=l1loss(recB,realB)#loss_rec(recB,realB)
                            
        g_loss=error_genAB+error_genBA+10*rec_errorA+10*rec_errorB
        #error_genAB.backward(retain_graph=True)
        #error_genBA.backward()
        #rec_errorA.backward()
        #rec_errorB.backward()
        g_loss.backward()#retain_graph=True)
        
        gen_optim.step()
                            
                            
        #########training discrminators
        dis_optim.zero_grad()
        
        
        fakeB=genAB(realA)
        fakeA=genBA(realB)
        recA=genBA(fakeB)
        recB=genAB(fakeA)
        dis_real_labB=discAB(realB)
        dis_fake_labB=discAB(fakeB)
        dis_real_labA=discBA(realA)
        dis_fake_labA=discBA(fakeA)
        dis_rec_labA=discBA(recA)
        dis_rec_labB=discAB(recB)

        
        
        #rec_errorA=loss_mse(dis_real_labB,Variable(torch.ones(realA.shape[0],1,30,30).to(cuda)))
        error_discAB_with_realB=loss_mse(dis_real_labB,Variable(torch.ones(realA.shape[0],1,30,30).to(cuda)))
        error_discAB_with_fakeB=loss_mse(dis_fake_labB,Variable(torch.zeros(realA.shape[0],1,30,30).to(cuda)))
        error_discBA_with_realA=loss_mse(dis_real_labA,Variable(torch.ones(realA.shape[0],1,30,30).to(cuda)))
        error_discBA_with_fakeA=loss_mse(dis_fake_labA,Variable(torch.zeros(realA.shape[0],1,30,30).to(cuda)))
        
        disc_loss=(error_discAB_with_realB+error_discAB_with_fakeB)/2+(error_discBA_with_realA+error_discBA_with_fakeA)/2
        disc_loss.backward()
        #error_discAB_with_fakeB.backward()
        #error_discBA_with_realB.backward()
        #error_discBA_with_realB.backward()
        if count%10==0:
            writer.add_image("realA",vutils.make_grid(realA,padding=2,normalize=True),count)
            writer.add_image("realB",vutils.make_grid(realB,padding=2,normalize=True),count)
            writer.add_image("fakeA",vutils.make_grid(fakeA,padding=2,normalize=True),count)
            writer.add_image("fakeB",vutils.make_grid(fakeB,padding=2,normalize=True),count)
            writer.add_image("recA",vutils.make_grid(recA,padding=2,normalize=True),count)
            writer.add_image("recB",vutils.make_grid(recB,padding=2,normalize=True),count)
        
        dis_optim.step()
        
        count=count+1
        print("epoch:{}  disc_loss:{}  gen_loss:{} recA Loss:{} recB Loss:{}".format(epoch,disc_loss,g_loss,rec_errorA
                                                                                     ,rec_errorB))

epoch:0  disc_loss:1.2698535919189453  gen_loss:15.986528396606445 recA Loss:0.5341123938560486 recB Loss:0.8394347429275513
epoch:0  disc_loss:6.763688564300537  gen_loss:14.379685401916504 recA Loss:0.5383883714675903 recB Loss:0.6883654594421387
epoch:0  disc_loss:1.1291406154632568  gen_loss:12.913152694702148 recA Loss:0.44804084300994873 recB Loss:0.6013553142547607
epoch:0  disc_loss:1.0378566980361938  gen_loss:10.563584327697754 recA Loss:0.39470452070236206 recB Loss:0.514710545539856
epoch:0  disc_loss:0.88199383020401  gen_loss:10.11788558959961 recA Loss:0.4148155152797699 recB Loss:0.43556198477745056
epoch:0  disc_loss:0.9612641334533691  gen_loss:9.483439445495605 recA Loss:0.3220863938331604 recB Loss:0.4478358030319214
epoch:0  disc_loss:0.8672472238540649  gen_loss:9.517078399658203 recA Loss:0.3531592786312103 recB Loss:0.4751320481300354
epoch:0  disc_loss:0.7682315707206726  gen_loss:7.816676139831543 recA Loss:0.2826690077781677 recB Loss:0.38349077105522156
epoc

epoch:0  disc_loss:0.688866376876831  gen_loss:4.765714645385742 recA Loss:0.1926005482673645 recB Loss:0.14415715634822845
epoch:0  disc_loss:1.1747729778289795  gen_loss:5.699143409729004 recA Loss:0.2105371356010437 recB Loss:0.14405730366706848
epoch:0  disc_loss:0.6846098899841309  gen_loss:4.671820640563965 recA Loss:0.2088434100151062 recB Loss:0.14250412583351135
epoch:0  disc_loss:0.5167863368988037  gen_loss:3.8801841735839844 recA Loss:0.16213171184062958 recB Loss:0.11040191352367401
epoch:0  disc_loss:0.8334227204322815  gen_loss:4.510922431945801 recA Loss:0.19368009269237518 recB Loss:0.11668373644351959
epoch:0  disc_loss:0.7300480604171753  gen_loss:4.789059638977051 recA Loss:0.22466211020946503 recB Loss:0.12952764332294464
epoch:0  disc_loss:0.4897916913032532  gen_loss:3.93502140045166 recA Loss:0.14698795974254608 recB Loss:0.14609724283218384
epoch:0  disc_loss:0.36844921112060547  gen_loss:4.272655487060547 recA Loss:0.1451374739408493 recB Loss:0.16523453593254

epoch:1  disc_loss:0.34357139468193054  gen_loss:3.5250916481018066 recA Loss:0.14431197941303253 recB Loss:0.11758366227149963
epoch:1  disc_loss:0.4006862938404083  gen_loss:3.843111515045166 recA Loss:0.18737724423408508 recB Loss:0.12333512306213379
epoch:1  disc_loss:0.4246136546134949  gen_loss:3.8311569690704346 recA Loss:0.15515786409378052 recB Loss:0.1654137372970581
epoch:1  disc_loss:0.3443676829338074  gen_loss:3.8009486198425293 recA Loss:0.15380333364009857 recB Loss:0.12484864890575409
epoch:1  disc_loss:0.357012003660202  gen_loss:3.175370931625366 recA Loss:0.1333254873752594 recB Loss:0.10720279067754745
epoch:1  disc_loss:0.3654448091983795  gen_loss:3.9046711921691895 recA Loss:0.1993635594844818 recB Loss:0.09287446737289429
epoch:1  disc_loss:0.38086622953414917  gen_loss:4.156851768493652 recA Loss:0.15719957649707794 recB Loss:0.1687217354774475
epoch:1  disc_loss:0.3759879469871521  gen_loss:3.859376907348633 recA Loss:0.18605002760887146 recB Loss:0.105464421

epoch:1  disc_loss:0.35212433338165283  gen_loss:4.652298927307129 recA Loss:0.18134531378746033 recB Loss:0.18922059237957
epoch:1  disc_loss:0.424760103225708  gen_loss:3.9596400260925293 recA Loss:0.18944361805915833 recB Loss:0.14488206803798676
epoch:2  disc_loss:0.322955459356308  gen_loss:6.649551868438721 recA Loss:0.3335305452346802 recB Loss:0.21012096107006073
epoch:2  disc_loss:0.37518754601478577  gen_loss:6.3519768714904785 recA Loss:0.3078348636627197 recB Loss:0.21584591269493103
epoch:2  disc_loss:0.35975518822669983  gen_loss:4.981622219085693 recA Loss:0.1999487727880478 recB Loss:0.19630688428878784
epoch:2  disc_loss:0.29731523990631104  gen_loss:5.1266679763793945 recA Loss:0.2954504191875458 recB Loss:0.13273286819458008
epoch:2  disc_loss:0.24879270792007446  gen_loss:4.679754257202148 recA Loss:0.18111205101013184 recB Loss:0.13117454946041107
epoch:2  disc_loss:0.293379545211792  gen_loss:4.929862976074219 recA Loss:0.17877721786499023 recB Loss:0.210237234830

epoch:2  disc_loss:0.18953777849674225  gen_loss:5.522010803222656 recA Loss:0.22799381613731384 recB Loss:0.12949153780937195
epoch:2  disc_loss:0.14224621653556824  gen_loss:4.489882469177246 recA Loss:0.18422441184520721 recB Loss:0.1146937757730484
epoch:2  disc_loss:0.2426024228334427  gen_loss:4.425002574920654 recA Loss:0.1811741590499878 recB Loss:0.1061786636710167
epoch:2  disc_loss:0.27191469073295593  gen_loss:4.548879146575928 recA Loss:0.19132515788078308 recB Loss:0.13533243536949158
epoch:2  disc_loss:0.2280293107032776  gen_loss:5.029030799865723 recA Loss:0.23012518882751465 recB Loss:0.11441312730312347
epoch:2  disc_loss:0.2949832081794739  gen_loss:4.372652053833008 recA Loss:0.17431943118572235 recB Loss:0.15990953147411346
epoch:2  disc_loss:0.4393935799598694  gen_loss:5.016482353210449 recA Loss:0.16796791553497314 recB Loss:0.1311418116092682
epoch:2  disc_loss:0.3944808542728424  gen_loss:3.884403705596924 recA Loss:0.17624089121818542 recB Loss:0.09262441098

...

epoch:97  disc_loss:0.16579657793045044  gen_loss:3.132890224456787 recA Loss:0.09612860530614853 recB Loss:0.053335871547460556
epoch:97  disc_loss:0.19499464333057404  gen_loss:3.5934994220733643 recA Loss:0.0980609804391861 recB Loss:0.04656827449798584
epoch:97  disc_loss:0.19798117876052856  gen_loss:3.7014198303222656 recA Loss:0.1061127707362175 recB Loss:0.049873024225234985
epoch:97  disc_loss:0.13836202025413513  gen_loss:4.413548469543457 recA Loss:0.10020583868026733 recB Loss:0.08520309627056122
epoch:97  disc_loss:0.15044018626213074  gen_loss:3.486809492111206 recA Loss:0.10952679812908173 recB Loss:0.04663744568824768
epoch:97  disc_loss:0.26260605454444885  gen_loss:3.8422341346740723 recA Loss:0.09646926820278168 recB Loss:0.04197940230369568
epoch:97  disc_loss:0.231615349650383  gen_loss:3.945514678955078 recA Loss:0.12970656156539917 recB Loss:0.05054701492190361
epoch:97  disc_loss:0.3473343849182129  gen_loss:2.710038185119629 recA Loss:0.11942094564437866 recB L

epoch:97  disc_loss:0.12926217913627625  gen_loss:3.5494232177734375 recA Loss:0.11823408305644989 recB Loss:0.05581236630678177
epoch:97  disc_loss:0.230610191822052  gen_loss:3.835064172744751 recA Loss:0.10754408687353134 recB Loss:0.04766293987631798
epoch:97  disc_loss:0.2316640466451645  gen_loss:3.155491828918457 recA Loss:0.10974925756454468 recB Loss:0.05564727634191513
epoch:97  disc_loss:0.2246154546737671  gen_loss:2.625728130340576 recA Loss:0.08949418365955353 recB Loss:0.033330559730529785
epoch:97  disc_loss:0.1492769718170166  gen_loss:3.1814754009246826 recA Loss:0.11889632791280746 recB Loss:0.03503444418311119
epoch:97  disc_loss:0.34622883796691895  gen_loss:3.498823881149292 recA Loss:0.11111374944448471 recB Loss:0.05038950592279434
epoch:97  disc_loss:0.3222563564777374  gen_loss:2.7834815979003906 recA Loss:0.1061299666762352 recB Loss:0.04371947795152664
epoch:97  disc_loss:0.16336789727210999  gen_loss:3.758289337158203 recA Loss:0.0909992977976799 recB Loss:

epoch:98  disc_loss:0.1857641339302063  gen_loss:2.8440799713134766 recA Loss:0.0917193740606308 recB Loss:0.03412599861621857
epoch:98  disc_loss:0.17776456475257874  gen_loss:4.3303093910217285 recA Loss:0.12102752178907394 recB Loss:0.08264486491680145
epoch:98  disc_loss:0.19641944766044617  gen_loss:3.5245275497436523 recA Loss:0.1018497571349144 recB Loss:0.05543778836727142
epoch:98  disc_loss:0.1643809676170349  gen_loss:3.5806705951690674 recA Loss:0.09007315337657928 recB Loss:0.05766836926341057
epoch:98  disc_loss:0.19182507693767548  gen_loss:3.201017141342163 recA Loss:0.08777458965778351 recB Loss:0.040429189801216125
epoch:98  disc_loss:0.22920215129852295  gen_loss:3.924142360687256 recA Loss:0.08635620772838593 recB Loss:0.048102736473083496
epoch:98  disc_loss:0.13168776035308838  gen_loss:3.801124095916748 recA Loss:0.10336869210004807 recB Loss:0.0799405574798584
epoch:98  disc_loss:0.2704848349094391  gen_loss:3.765641450881958 recA Loss:0.11875196546316147 recB L

epoch:99  disc_loss:0.2694370150566101  gen_loss:3.496973752975464 recA Loss:0.09550894796848297 recB Loss:0.05605218559503555
epoch:99  disc_loss:0.19905057549476624  gen_loss:3.0801730155944824 recA Loss:0.09202586114406586 recB Loss:0.05459865927696228
epoch:99  disc_loss:0.24557584524154663  gen_loss:3.082698345184326 recA Loss:0.09274260699748993 recB Loss:0.051695507019758224
epoch:99  disc_loss:0.1527349352836609  gen_loss:3.5179638862609863 recA Loss:0.09160976856946945 recB Loss:0.055417001247406006
epoch:99  disc_loss:0.266873836517334  gen_loss:4.082244396209717 recA Loss:0.1357572078704834 recB Loss:0.06792234629392624
epoch:99  disc_loss:0.1500348001718521  gen_loss:3.3789799213409424 recA Loss:0.08663487434387207 recB Loss:0.045569196343421936
epoch:99  disc_loss:0.1822829246520996  gen_loss:3.900160789489746 recA Loss:0.12033210694789886 recB Loss:0.048682019114494324
epoch:99  disc_loss:0.15572412312030792  gen_loss:3.7686054706573486 recA Loss:0.09780025482177734 recB 

epoch:99  disc_loss:0.35589390993118286  gen_loss:2.3044469356536865 recA Loss:0.08874282240867615 recB Loss:0.04496624320745468
epoch:99  disc_loss:0.20847606658935547  gen_loss:3.978609085083008 recA Loss:0.09114007651805878 recB Loss:0.035298071801662445
epoch:99  disc_loss:0.2891272008419037  gen_loss:4.045035362243652 recA Loss:0.1267496794462204 recB Loss:0.04963940382003784
epoch:99  disc_loss:0.3053439259529114  gen_loss:2.6135520935058594 recA Loss:0.10315313935279846 recB Loss:0.04481522738933563
epoch:99  disc_loss:0.26821768283843994  gen_loss:4.23602819442749 recA Loss:0.1113080233335495 recB Loss:0.03950175642967224
epoch:99  disc_loss:0.16336677968502045  gen_loss:3.6351661682128906 recA Loss:0.10648583620786667 recB Loss:0.037355102598667145
epoch:99  disc_loss:0.22872227430343628  gen_loss:3.07414174079895 recA Loss:0.1005467027425766 recB Loss:0.04972665756940842
epoch:99  disc_loss:0.20860370993614197  gen_loss:3.4773290157318115 recA Loss:0.11132305860519409 recB Lo

In [ ]:
dis_fake_labB.shape

In [23]:
rec_errorB

tensor(109.0262, device='cuda:2', grad_fn=<L1LossBackward>)

In [86]:
real_data['A'].shape

torch.Size([4, 3, 256, 256])

In [89]:
realA.shape

torch.Size([4, 3, 256, 256])